In [1]:
import sys
sys.path.insert(0, "/home/uih/JYL/Programs/YOLO/")
import torch
from models import Yolov5Small

In [2]:
class_num = 20

In [3]:
yolo = Yolov5Small(3, class_num)
my_state_dict = yolo.state_dict()

In [ ]:
my_total_k = []
my_total_v = []
my_focus_layer = []
my_stage_1_conv = []
for i, (k, v) in enumerate(my_state_dict.items()):
    print(f'{i}: {k}\t\t{v.shape}')
    my_total_k.append(k)
    my_total_v.append(v)
    if "focus" in k:
        my_focus_layer.append(v)
        continue
    if "stage1_conv" in k:
        my_stage_1_conv.append(v)
        

In [5]:
v5_state_dict = torch.load('/home/uih/JYL/Programs/yolov5_version6/torch_state_dict_yolos.pth', map_location='cpu')

In [6]:
# v5_state_dict = v5_state_dict['model']

In [ ]:
v5_total_k, v5_total_v = [], []
v5_focus_layer = []
v5_stage_1_conv = []
for i, (k, v) in enumerate(v5_state_dict.items()):
    if 'anchor' not in k:
        v5_total_k.append(k)
        v5_total_v.append(v)
        print(f'{i}: {k}\t\t{v.shape}')
    if "model.0" in k:
        v5_focus_layer.append(v)
    if "model.1" in k:
        v5_stage_1_conv.append(v)

In [8]:
len(my_total_k), len(v5_total_k)

(348, 348)

In [ ]:
for i in range(len(my_total_k)):
    if my_total_v[i].shape != v5_total_v[i].shape:
        print(i)

## load prarmeters

In [ ]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 342:
        continue
    if isinstance(m, torch.nn.Conv2d):
        m.weight.data = total[i]
        print(m.weight.data.shape, total[i].shape)
        i += 1
        if m.bias is not None:
            m.bias.data = total[i]
            i += 1
        
    if isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data = total[i]
        i += 1
        m.bias.data = total[i]
        i += 1
        m.running_mean = total[i]
        i += 1
        m.running_var = total[i]
        i += 1
        m.num_batches_tracked = total[i]
        i += 1

In [11]:
i

342

In [12]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 342:
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        if m.bias is not None:
            assert (m.bias.data == total[i]).all()
            i += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        assert (m.bias.data == total[i]).all()
        i += 1
        assert (m.running_mean == total[i]).all()
        i += 1
        assert (m.running_var == total[i]).all()
        i += 1
        assert (m.num_batches_tracked == total[i]).all()
        i += 1

In [13]:
my_output_state = yolo.state_dict()
torch.save({'model_state_dict': my_output_state}, "/home/uih/JYL/Programs/YOLO_ckpts/yolov5_small_for_voc.pth")